# Perform VADER Sentiment on Bitcoin and Litecoin to determine news stories sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mrnagleJR97/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
crypto_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Bitcoin: {crypto_headlines['totalResults']}")

# Show sample article
crypto_headlines["articles"][0]



Total articles about Bitcoin: 6662


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Sarah Perez',
 'title': 'PayPal expands the ability to buy, hold and sell cryptocurrency to the U.K.',
 'description': 'PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to select between four types of cryptocurrencies — including Bi…',
 'url': 'http://techcrunch.com/2021/08/23/paypal-expands-the-ability-to-buy-hold-and-sell-cryptocurrency-to-the-u-k/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2020/11/GettyImages-887657568.jpg?w=600',
 'publishedAt': '2021-08-23T13:49:45Z',
 'content': 'PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to … [+4420 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
crypto_sentiments = []

for article in crypto_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        crypto_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
crypto_df = pd.DataFrame(crypto_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
crypto_df = crypto_df[cols]

crypto_df.head()



,date,text,compound,positive,negative,neutral
0,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.0,0.902
1,2021-09-07,A recently-installed Bitcoin ATM.\r\n\n \n\n A...,0.1779,0.052,0.0,0.948
2,2021-09-07,The government of El Salvador purchased at lea...,0.1280,0.046,0.0,0.954
3,2021-08-19,Retailers are increasingly accepting cryptocur...,0.6187,0.153,0.0,0.847
4,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.0,0.839


In [7]:
# Get descriptive stats from the DataFrame
crypto_df.describe()



,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.104099,0.062240,0.033320,0.904430
std,0.358923,0.060281,0.057001,0.081231
min,-0.893400,0.000000,0.000000,0.622000
25%,0.000000,0.000000,0.000000,0.849250
50%,0.070800,0.050500,0.000000,0.913000
75%,0.340000,0.096500,0.055750,0.959250
max,0.811600,0.213000,0.312000,1.000000


In [8]:
crypto_df.to_csv('./input_data/btc_sentiment.csv')